In [1]:
!curl https://zenodo.org/record/3974431/files/vanilla_bert_tiny_on_MSMARCO.tar.gz >> vanilla_bert_tiny_on_MSMARCO.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 42.2M  100 42.2M    0     0  7181k      0  0:00:06  0:00:06 --:--:-- 9226k


In [2]:
!tar -zxvf vanilla_bert_tiny_on_MSMARCO.tar.gz

bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/
bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/model.ckpt-1600000.data-00000-of-00001
bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/model.ckpt-1600000.meta
bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/model.ckpt-1600000.index


In [27]:
!curl https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-2_H-128_A-2.zip >> bert.zip
!unzip bert.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.7M  100 15.7M    0     0  6607k      0  0:00:02  0:00:02 --:--:-- 6610k
Archive:  bert.zip
  inflating: bert_model.ckpt.data-00000-of-00001  
  inflating: bert_config.json        
replace vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [8]:
import pyterrier as pt
if not pt.started():
    pt.init(version="snapshot")
    
import tensorflow as tf

from pyterrier.transformer import TransformerBase

from run_reranking import model_fn_builder, input_fn_builder
from bert.modeling import BertConfig

In [51]:
class PARADEPipeline(TransformerBase):
    def __init__(self, aggregation_method):
        self.aggregation_method = aggregation_method #'cls_max',  'cls_avg', 'cls_attn' or 'cls_transformer'
        
        
        
    def transform(self, queries_and_docs):
        run_config = tf.estimator.tpu.RunConfig(
            cluster=None,
            model_dir=None,
            save_checkpoints_steps=1000,
            keep_checkpoint_max=1,
            tpu_config=None)
            
        '''
        tpu_config is set to None since we aren't using a tpu, but in case we actually need this
        i'll just keep this commented out instead of deleting it
            
            tpu_config=tf.estimator.tpu.TPUConfig(
                iterations_per_loop=1000,
                num_shards=8,
                per_host_input_for_training=is_per_host))
        '''
        
        model_fn = model_fn_builder(
            bert_config=BertConfig.from_json_file('bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/bert_config.json'),
            num_labels=2,
            init_checkpoint='bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/model.ckpt-1600000',
            learning_rate=5e-5,
            num_train_steps=None,
            num_warmup_steps=None,
            use_tpu=False,
            use_one_hot_embeddings=False,
            aggregation_method=self.aggregation_method,
            pretrained_model='bert',
            from_distilled_student=False)
            
        estimator = tf.estimator.tpu.TPUEstimator(
            use_tpu=False,
            model_fn=model_fn,
            config=run_config,
            train_batch_size=32,
            eval_batch_size=32,
            predict_batch_size=32)
            
        eval_input_fn = input_fn_builder(
            dataset_path='index_vaswani/data.meta.idx',
            max_num_segments_perdoc=8,
            max_seq_length=128,
            is_training=False)
            
        result = estimator.predict(input_fn=eval_input_fn, yield_single_examples=True)
        
        results = []
        for item in result:
            results.append((item["log_probs"], item["label_ids"]))
            

In [52]:
pipeline = PARADEPipeline(aggregation_method='cls_max')

vaswani  = pt.datasets.get_dataset("vaswani")
vaswani.get_corpus()

pipeline.transform(vaswani)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp3z37k_8x', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f73a41992e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores_per_replica=None,

INFO:tensorflow:prediction_loop marked as finished


DataLossError: corrupted record at 0
	 [[node IteratorGetNext (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'IteratorGetNext':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.6/dist-packages/tornado/platform/asyncio.py", line 149, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.6/asyncio/base_events.py", line 438, in run_forever
    self._run_once()
  File "/usr/l